In [1]:
# pip install fastbook

In [2]:
from fastai.vision.all import *

path= untar_data(URLs.MNIST_SAMPLE)
threes= (path/'train'/'3').ls().sorted()
sevens= (path/'train'/'7').ls().sorted()

three_tensors= [tensor(Image.open(o)) for o in threes]
seven_tensors= [tensor(Image.open(o)) for o in sevens]

stacked_threes= torch.stack(three_tensors).float()/255
stacked_sevens= torch.stack(seven_tensors).float()/255

train_x = torch.cat([stacked_threes, stacked_sevens]).view(-1, 28 * 28)

def mnist_loss(predictions, targets):
  return torch.where(targets==1, 1-predictions, predictions).mean()

valid_3_tens= torch.stack([tensor(Image.open(o)) for o in (path/'valid'/'3').ls()])
valid_7_tens= torch.stack([tensor(Image.open(o)) for o in (path/'valid'/'7').ls()])

valid_3_tens= valid_3_tens.float()/255
valid_7_tens= valid_7_tens.float()/255

In [3]:
# #plot_function 찾기 실패, 따라서 커뮤니티의 소스코드 불러옴
# def plot_function(f, tx=None, ty=None, title=None, min=-2, max=2, figsize=(6,4)):
#     x = torch.linspace(min,max, 1)
#     fig,ax = plt.subplots(figsize=figsize)
#     ax.plot(x,f(x))
#     if tx is not None: ax.set_xlabel(tx)
#     if ty is not None: ax.set_ylabel(ty)
#     if title is not None: ax.set_title(title)

# 4.5.1 시그모이드
##### 이전 노트에서 정의한 mnist_loss 함수는 예측이 항상 0과 1 사잇값이라고 가정하는 문제가 있다. 하지만, 값이 0과 1 사이가 되도록 강제하여 호가실히 해야한다.
### 항상 0과 1 사이의 숫자를 출력하는 시그모이드 함수는 다음과 같이 정의된다.

In [4]:
def sigmoid(x): return 1/(1+torch.exp(-x))

In [5]:
# plot_function(torch.sigmoid, title="Sigmoid", min=-4, max=4)

### 시그모이드 함수는
##### 입력값은 음수부터 양수까지 제한은 없지만, 출력값은 '0과 1 사이'이다.
##### ! 또한, 오직 '증가**만**' 하는 부드로운 곡선이다. 따라서, SGD가 의미 있는 그레디언트를 더 쉽게 찾도록 해준다.
### 따라서, 입력된 값(예측값)에 시그모이드가 적용되도록 mnist_loss 함수를 갱신하도록 하자.



In [6]:
def mnist_loss(predictions, targets):
  predictions= predictions.sigmoid()
  return torch.where(targets== 1, 1-predictions, predictions).mean()

# 4.5.2 SGD와 미니배치
## 최적화 단계
##### * 가중치를 갱신하는 단계
##### ** 하나 이상의 데이터에 대한 손실을 계산해야한다.
### 얼마나 많은 데이터가 필요?
#### - 전체 데이터를 계산
##### 손실 계산 후 평균 -> 시간이 오랠걸린다.
#### - 단일 데이터만을 계산
##### 많은 정보 활용 불가 -> 부정확, 불안정 그래디언트 계산
### 따라서, 두 방법 모두 가중치 갱신에 문제를 겪는다.

## 절충안: 미니배치학습
##### 일정 개수의 데이터에 대한 손실의 평균을 계산한다.
##### * 배치 크기(batch size): 미니 배치에 포함된 데이터의 개수
### 따라서, 적당한 배치 크기를 고를 줄 알아야 한다.

## 파이토치와 fastai는
### 임의로 데이터셋을 뒤섞은 다음 미니배치를 만드는 DataLoader 클래스를 제공한다.

In [7]:
coll= range(15)
dl= DataLoader(coll, batch_size=5, shuffle= True)
list(dl)

[tensor([13,  1, 11, 10,  0]),
 tensor([12,  2, 14,  7,  8]),
 tensor([5, 3, 4, 6, 9])]

### !! 모델 학습에는 임의의 파이썬 컬렉션을 사용해서는 안된다 !!
## 대신
### 독립변수와 종속변수(모델의 입력과 타깃(레이블))를 포함한 컬렉션이 필요하다.

In [8]:
ds= L(enumerate(string.ascii_lowercase))
ds

(#26) [(0, 'a'),(1, 'b'),(2, 'c'),(3, 'd'),(4, 'e'),(5, 'f'),(6, 'g'),(7, 'h'),(8, 'i'),(9, 'j')...]

### L(enumerate(string))
##### L(): fastai에서 제공하는 L클래스 객체, L은 파이썬에 내장된 list를 확장하여 추가 기능을 얹은 클래스이다.(p.185참고)
##### enumerate(obj): 입력값의 내용 순서화, (순서, 입렵값의 내용)튜플형식으로 반환한다.

In [9]:
dl= DataLoader(ds, batch_size=6, shuffle=True)
list(dl)

[(tensor([ 2, 17, 14,  4, 24, 12]), ('c', 'r', 'o', 'e', 'y', 'm')),
 (tensor([ 0, 15, 11, 23,  1,  5]), ('a', 'p', 'l', 'x', 'b', 'f')),
 (tensor([20, 13,  8,  6, 19,  9]), ('u', 'n', 'i', 'g', 't', 'j')),
 (tensor([22, 21, 18, 16, 10,  7]), ('w', 'v', 's', 'q', 'k', 'h')),
 (tensor([ 3, 25]), ('d', 'z'))]

# 4.6 모든 것을 한자리에
### 이제 매 에포크에 구현되어야 할 과정의 코드를 작성해보자.

In [10]:
# for x, y in dl:
#   pred= model(x)
#   loss= loss_func(pred, y)
#   loss.backward()
#   parameters -= parameters.grad * lr

### 우선 전에 작성했던 변수와 함수들부터 다시 작성

In [11]:
train_x= torch.cat([stacked_threes, stacked_sevens]).view(-1, 28*28)
train_y= tensor([1]*len(threes) + [0]*len(sevens)).unsqueeze(1)

dset= list(zip(train_x, train_y))

valid_x= torch.cat([valid_3_tens, valid_7_tens]).view(-1, 28*28)
valid_y= tensor([1]*len(valid_3_tens) + [0]*len(valid_7_tens)).unsqueeze(1)

valid_dset= list(zip(valid_x, valid_y))

def init_params(size, std=1.0): return (torch.randn(size)*std).requires_grad_()

def linear1(xb): return xb@weights + bias

In [12]:
weights= init_params((28*28, 1))
bias= init_params(1)

dl= DataLoader(dset, batch_size= 256)
xb, yb= first(dl)
xb.shape, yb.shape

(torch.Size([256, 784]), torch.Size([256, 1]))

In [13]:
valid_dl= DataLoader(valid_dset, batch_size=256)

In [14]:
batch= train_x[:4]
batch.shape

torch.Size([4, 784])

In [15]:
preds= linear1(batch)
preds

tensor([[ 6.0427],
        [ 1.2961],
        [-0.5092],
        [-0.2642]], grad_fn=<AddBackward0>)

In [16]:
loss= mnist_loss(preds, train_y[:4])
loss

tensor(0.3519, grad_fn=<MeanBackward0>)

In [17]:
loss.backward()
weights.grad.shape, weights.grad.mean(), bias.grad

(torch.Size([784, 1]), tensor(-0.0253), tensor([-0.1628]))

In [18]:
def calc_grad(xb, yb, model):
  preds= model(xb)
  loss= mnist_loss(preds, yb)
  loss.backward()

##### 함수로 만든 버전을 검사해보자.

In [21]:
calc_grad(batch, train_y[:4], linear1)
weights.grad.mean(), bias.grad

(tensor(-0.1012), tensor([-0.6512]))

##### 한번 더 호출해보면?

In [22]:
calc_grad(batch, train_y[:4], linear1)
weights.grad.mean(), bias.grad

(tensor(-0.1265), tensor([-0.8140]))

### 그래디언트가 변화했다!?!?
##### loss.backward는 지금 계산된 손실의 그레디언트를
##### 앞서 계산된 그레디언트에 '더하기' 때문이다.
##### 따라서, 이전의 그레디언트를 0으로 설정해줘야 한다.

In [23]:
weights.grad.zero_()
bias.grad.zero_();

### .zero_()
#### 제자리연산자: '_'
##### 파이토치 제공, 메소드 중 이름의 마지막에 밑줄이 포함된 것은
##### 해당 객체를 제자리에서 조작한다.

### 이제, 그레디언트 및 학습률에 기반해 가중치와 편향을 갱신하는 단계만 남음.
### 여기서도, 파이토치가 그레디언트를 계산하지 못하도록 조치해야 한다.
### 텐서의 data 속성에 값을 할당하면, 파이토치는 해당 단계에서 그레디언트를 계산하지 않는다.

In [24]:
# 간단한 학습 루프
def train_epoch(model, lr, params):
  for xb, yb in dl:
    calc_grad(xb, yb, model)
    for p in params:
      p.data -= p.grad * lr
      p.grad.zero_()

### 또한 검증용 데이터셋으로 정확도를 확인하여 얼마나 이뤄지는지를 확인해야 한다.
##### 예측 출력이 0.5보다 큰지를 확인하여 3과 7중 무엇을 의미하는지 판단할 수 있다. 따라서, 다음처럼 각 데이터에 대한 정확도를 계산할 수 있다.

In [25]:
(preds>0.5).float() == train_y[:4]

tensor([[ True],
        [ True],
        [False],
        [False]])

##### 이 방식으로 배치 단위의 평균 정확도를 계산하는 함수를 만들 수 있다.

In [26]:
def batch_accuracy(xb, yb):
  preds= xb.sigmoid()
  correct= (preds>0.5) == yb
  return correct.float().mean()

In [27]:
batch_accuracy(linear1(batch), train_y[:4])

tensor(0.5000)

##### 그리고 검증용 데이터셋의 모든 배치에 batch_accuracy 함수를 적용해서 얻은 결과들의 평균을 구해보도록 하자.

In [28]:
def validate_epoch(model):
  accs= [batch_accuracy(model(xb), yb) for xb, yb in valid_dl]
  return round(torch.stack(accs).mean().item(), 4)

validate_epoch(linear1)

0.3874

##### 여기까지가 학습을 진행하기 위한 출발점.
### 이번에는 한 에포크 동안 모델을 학습시킨 다음 정확도가 개선도니ㅡㄴ지를 호가인해 보도록 하자.

In [29]:
lr= 1.
params= weights, bias
train_epoch(linear1, lr, params)
validate_epoch(linear1)

0.5735

##### epoch 여러번 더 반복

In [30]:
for i in range(20):
  train_epoch(linear1, lr, params)
  print(validate_epoch(linear1), end='')

0.6820.84760.9120.93110.94180.94910.95060.95350.95690.95980.96130.96280.96330.96330.96380.96430.96430.96470.96520.9653

##### 다음으로는 SGD 단계를 포장하여, 객체로서 다룰 수 있도록 만들어보도록 하자.
### 파이토치는 이런 객체를 "옵티마이저"라고 한다.